In [1]:
import os
import torch
from utils import videos_to_frames, extract_faces, interpolate_missing_frames, resize_excess, pad_resize, save_as_tensor
from torch.utils.data import DataLoader

In [2]:
from model import FakeNet
model = FakeNet()
model = torch.nn.DataParallel(model)
print(model)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

DataParallel(
  (module): FakeNet(
    (feature_extract): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (cnn_relu): ReLU()
    (manipulator): MotionMag(
      (subs): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu1): ReLU()
      (conv_mask1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu2): ReLU()
      (conv_mask2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu3): ReLU()
      (cnn_h): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu4): ReLU()
    )
    (temporal_extractor): LSTM(10500, 48, batch_first=True)
    (linear1): Linear(in_features=14352, out_features=1, bias=True)
  )
)


In [3]:
load_name = os.path.join('fake_class2_29_checkpoint.pth.tar')
loaded_model = torch.load(load_name, map_location=torch.device('cpu'))
model.load_state_dict(loaded_model['state_dict'])
#optimizer.load_state_dict(loaded_model['optimizer'])
start_epoch = loaded_model['epoch']

### Run following lines to test on full data

In [7]:
%env KAGGLE_USERNAME=""
%env KAGGLE_KEY=""

['aabdogagch']


In [ ]:
!kaggle competitions download -c deepfake-detection-challenge

### Following lines preprocess the data
demo_data_path should be the name of folder which encloses videos

In [21]:
demo_data_path = "demo_data"
videos_to_frames(demo_data_path)
extract_faces(demo_data_path)
interpolate_missing_frames(demo_data_path)
resize_excess(demo_data_path)
pad_resize(demo_data_path)

aepuluzrdy->298


### Following line saves images as one tensor

In [ ]:
save_as_tensor(demo_data_path)

In [31]:
from data_loader import FakeDetectDataset
dataset = FakeDetectDataset(demo_data_path)
data_loader_params = {'dataset': dataset, 'batch_size': 1, 'shuffle': False, 'sampler': None,
                        'batch_sampler': None, 'num_workers': 0, 'collate_fn': None}
eval_loader = DataLoader(**data_loader_params)

torch.Size([399, 299, 3, 70, 50])


In [32]:
model.eval()
with torch.no_grad():
    for (batch_no, sample) in enumerate(eval_loader):
            inp = sample[0]
            label = torch.tensor(sample[1], dtype=torch.float)
            output = model(inp)
            output = output.squeeze()
            number_to_probs = lambda class_int: "REAL prob: {:.2f} FAKE probability: {:.2f}".format(torch.sigmoid(output), 1-torch.sigmoid(output).item()) 
            number_to_class = lambda class_int: "REAL" if  class_int > 0.5 else "FAKE"
            print(output)
            class_pred = torch.sigmoid(output)

            acc = torch.sum(class_pred == label) / len(label)
            print(f"Prediction = {number_to_probs(class_pred.item())}", end="\t")
            print(f"Actual = {number_to_class(label.item())}")

/var/folders/s3/4dnbkq8j50b22mmhg1h6wpr80000gn/T/ipykernel_80536/933792671.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(sample[1], dtype=torch.float)


tensor(-6.0015)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-6.5975)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(7.8611)
Prediction = REAL prob: 1.00 FAKE probability: 0.00	Actual = REAL
tensor(-8.3913)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-8.9898)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-8.9275)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-6.0201)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-7.8464)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-7.4918)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-6.3236)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-11.7653)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-5.6604)
Prediction = REAL prob: 0.00 FAKE probability: 1.00	Actual = FAKE
tensor(-8.8486)


KeyboardInterrupt: 